In [1]:
import numpy as np
import matplotlib.pyplot as plt
from deps import cv_core
import cv2
import glob
from deps.dobot_api import DobotApiDashboard, DobotApiMove

In [15]:
# Select the camera number , Generally from 0 Start
cap = cv2.VideoCapture(0)
# Set the parameters first , And then read the parameterss
for i in range(47):
    print("No.={} parameter={}".format(i,cap.get(i)))

cap.release()

No.=0 parameter=0.0
No.=1 parameter=-1.0
No.=2 parameter=-1.0
No.=3 parameter=640.0
No.=4 parameter=480.0
No.=5 parameter=30.0
No.=6 parameter=1448695129.0
No.=7 parameter=-1.0
No.=8 parameter=16.0
No.=9 parameter=0.0
No.=10 parameter=10.0
No.=11 parameter=32.0
No.=12 parameter=75.0
No.=13 parameter=0.0
No.=14 parameter=10.0
No.=15 parameter=5000.0
No.=16 parameter=1.0
No.=17 parameter=-1.0
No.=18 parameter=-1.0
No.=19 parameter=-1.0
No.=20 parameter=3.0
No.=21 parameter=3.0
No.=22 parameter=100.0
No.=23 parameter=4600.0
No.=24 parameter=-1.0
No.=25 parameter=-1.0
No.=26 parameter=-1.0
No.=27 parameter=-1.0
No.=28 parameter=-1.0
No.=29 parameter=-1.0
No.=30 parameter=-1.0
No.=31 parameter=-1.0
No.=32 parameter=1.0
No.=33 parameter=-1.0
No.=34 parameter=-1.0
No.=35 parameter=-1.0
No.=36 parameter=-1.0
No.=37 parameter=-1.0
No.=38 parameter=4.0
No.=39 parameter=-1.0
No.=40 parameter=-1.0
No.=41 parameter=-1.0
No.=42 parameter=200.0
No.=43 parameter=-1.0
No.=44 parameter=1.0
No.=45 parame

[ WARN:0@1529.844] global /io/opencv/modules/videoio/src/cap_v4l.cpp (902) open VIDEOIO(V4L2:/dev/video0): can't open camera by index


In [2]:
cap = cv2.VideoCapture(0)
#cap.set(cv2.CAP_PROP_WHITE_BALANCE_RED_V,10)
#cap.set(cv2.CAP_PROP_AUTO_WB, 0)

while(True):
    ret, frame = cap.read()
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

[ WARN:0@1.545] global /opt/conda/conda-bld/opencv-suite_1656606573658/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
libGL error: MESA-LOADER: failed to open radeonsi: /usr/lib/dri/radeonsi_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: radeonsi
libGL error: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: swrast


In [6]:
cap = cv2.VideoCapture(0)
cap = cv_core.set_res(cap, camera_res['1200'])
cv_core.video_test(cap)
cap.release()

[ WARN:0@24.604] global /opt/conda/conda-bld/opencv-suite_1656606573658/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


In [3]:
features_mm_to_pixels_dict =  {(382.76, -113.37): (499, 412), #bottom left
                                (225.27, 94.68): (240, 103), #top right
                                (386.5, 91.55): (492, 98), # bottom right
                                (221.25, -110.62): (248, 419)} # top left

tf_mtx = cv_core.compute_tf_mtx(features_mm_to_pixels_dict)

In [11]:
cap = cv2.VideoCapture(0)
cap = cv_core.set_res(cap, cv_core.camera_res_dict['1200'])
cont = cv_core.Contours()

while(True):
    ret, frame = cap.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    #ret,thresh = cv2.threshold(gray,125,255,cv2.THRESH_BINARY_INV)
    kernel = np.ones((3,3),np.uint8)
    #closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

    plot_img = frame.copy()
    # cv2.circle(plot_img, (a, b), r, (0, 255, 0), 2)
    # cv2.circle(plot_img, (a, b), 1, (255, 0, 0), 3)
    #with_contours = cv2.drawContours(plot_img, sorted_contours, -1,(0,255, 0),2)

    # for c in sorted_contours:
    #     # compute the center of the contour
    #     M = cv2.moments(c)
    #     cX = int(M["m10"] / M["m00"])
    #     cY = int(M["m01"] / M["m00"])
    #     # draw the contour and center of the shape on the image
    #     #cv2.drawContours(plot_img, [c], -1, (0, 255, 0), 2)
    #     cv2.circle(plot_img, (cX, cY), 2, (0, 0, 255), -1)
    #     cv2.putText(plot_img, f"{cX},{cY}", (cX - 20, cY - 20),
    #     cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    #out.write(with_contours)

    cv2.imshow('frame',opening)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
#out.release()
cv2.destroyAllWindows()

In [14]:
# images = sorted(glob.glob('camera_data/raw/*.png'))


# font = cv2.FONT_HERSHEY_SIMPLEX
  
# # org
# org = (50, 50)
  
# # fontScale
# fontScale = 1
   
# # Blue color in BGR
# color = (0, 255, 0)
  
# # Line thickness of 2 px
# thickness = 2

# for idx, image in enumerate(images):
#     img = cv2.imread(image)
#     img = cv2.putText(img, f'{idx}', org, font, 
#                    fontScale, color, thickness, cv2.LINE_AA)
#     cv2.imwrite(f'camera_data/marked/image{idx}.png', img)

In [40]:
cameraMatrix = np.load('./cam_matrices/cam_mtx.npy')
dist = np.load('./cam_matrices/dist.npy')
newCameraMatrix = np.load('./cam_matrices/newcam_mtx.npy')

cap = cv2.VideoCapture(0)
cap = cv_core.set_res(cap, camera_res['1200'])

while True:
    ret, frame = cap.read()
    
    frame = cv2.undistort(frame, cameraMatrix, dist, None, newCameraMatrix)
    cv2.namedWindow('frame', cv2.WINDOW_NORMAL)
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
            
cap.release()
cv2.destroyAllWindows()

In [3]:
cont = cv_core.Contours()
cap = cv2.VideoCapture(0)
cap = cv_core.set_res(cap, camera_res['768'])
centers = cont.wait_for_anchors(cap, show = True)
cap.release()

Qt: Session management error: Could not open network socket


In [5]:
dash = DobotApiDashboard('192.168.1.6', 29999)
move = DobotApiMove('192.168.1.6', 30003)

In [7]:
dash.ClearError()
dash.EnableRobot()

'0,{},EnableRobot();'

In [22]:
cameraMatrix = np.load('./cam_matrices/cam_mtx.npy')
dist = np.load('./cam_matrices/dist.npy')
newCameraMatrix = np.load('./cam_matrices/newcam_mtx.npy')

cap = cv2.VideoCapture(0)
cap = cv_core.set_res(cap, camera_res['1200'])

while True:
    ret, frame = cap.read()
    
    frame = cv2.undistort(frame, cameraMatrix, dist, None, newCameraMatrix)

    cv2.line(frame, (int(frame.shape[1]/2), 0), (int(frame.shape[1]/2), frame.shape[0]), (0, 255, 0), thickness=2)

    cv2.namedWindow('frame', cv2.WINDOW_NORMAL)
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

In [23]:
img = frame.copy()
cv2.imshow('frame',np.split(img,2, axis = 1)[1])
cv2.waitKey(0)
cv2.destroyAllWindows()

# New circle detector

In [14]:
cap = cv2.VideoCapture(0)
cap = cv_core.set_res(cap, (1024, 768))

while True:
    ret, frame = cap.read()

    # img = frame
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,11,5)
    # #_,img = cv2.threshold(img,127,255,cv.THRESH_BINARY)
    # kernel = np.ones((2,2),np.uint8)
    # img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
    # #img = cv2.Canny(img, 5, 70, 3)

    # blurred = cv2.blur(img, (7, 7))

    # detected_circles = cv2.HoughCircles(image=blurred,
    #                                     method=cv2.HOUGH_GRADIENT,
    #                                     dp=1.5,
    #                                     minDist=500,
    #                                     param1=100,
    #                                     param2=50,
    #                                     minRadius=50,
    #                                     maxRadius=300
    #                                     )

    # detected_circles = np.uint16(np.around(detected_circles))
    # pt = detected_circles[0][0]
    # a, b, r = pt
    # plot_img = frame.copy()
    # cv2.circle(plot_img, (a, b), r, (0, 255, 0), 2)
    # cv2.circle(plot_img, (a, b), 1, (255, 0, 0), 3)

    img = frame.copy()
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    blur = cv2.GaussianBlur(img,(3,3),0)
    ret, thresh = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

    #ret,thresh = cv2.threshold(img,127,255,cv2.THRESH_BINARY)
    kernel = np.ones((3,3),np.uint8)
    #closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    #opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    dilation = cv2.dilate(thresh,kernel,iterations = 3)

    blurred = cv2.blur(dilation, (7, 7))
    detected_circles = cv2.HoughCircles(image=blurred,
                                        method=cv2.HOUGH_GRADIENT,
                                        dp=1.2,
                                        minDist=500,
                                        param1=100,
                                        param2=50,
                                        minRadius=50,
                                        maxRadius=300
                                        )

    detected_circles = np.uint16(np.around(detected_circles))
    pt = detected_circles[0][0]
    a, b, r = pt
    plot_img = frame.copy()
    cv2.circle(plot_img, (a, b), r, (0, 255, 0), 2)
    cv2.circle(plot_img, (a, b), 1, (255, 0, 0), 3)

    cv2.imshow('frame', plot_img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

[ WARN:0@528.662] global /io/opencv/modules/videoio/src/cap_v4l.cpp (902) open VIDEOIO(V4L2:/dev/video0): can't open camera by index


AttributeError: 'NoneType' object has no attribute 'copy'

## New anchors

In [2]:
def sort_anchors(inp):
    maxx = sorted(np.squeeze(inp), key=lambda x: x[0])

    left = maxx[:2]
    right = maxx[-2:]

    l_sorty = sorted(left, key=lambda x: x[1])
    r_sorty = sorted(right, key=lambda x: x[1])

    return np.vstack((l_sorty, r_sorty))

In [11]:
cap = cv2.VideoCapture(0)
cap = cv_core.set_res(cap, (1600, 1200))
cv2.namedWindow('frame',  cv2.WINDOW_NORMAL)
cv2.resizeWindow('frame', 1348, 1011)

cameraMatrix = np.load('./cam_matrices/cam_mtx.npy')
dist = np.load('./cam_matrices/dist.npy')
newCameraMatrix = np.load('./cam_matrices/newcam_mtx.npy')

all_anchors = []

idx = 1
while True:
    ret, frame = cap.read()
    frame = cv2.undistort(frame, cameraMatrix, dist, None, newCameraMatrix)
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    detected_circles = cv2.HoughCircles(image=gray,
                                        method=cv2.HOUGH_GRADIENT,
                                        dp=1,
                                        minDist=30,
                                        param1=100,
                                        param2=40,
                                        minRadius=20,
                                        maxRadius=40
                                        )

    plot_img = frame.copy()
    if detected_circles is not None:
        circles = np.uint16(np.around(detected_circles))
        if np.squeeze(circles).shape[0] == 4:
            all_anchors.append(sort_anchors(circles))

        for i in circles[0,:]:
            center = (i[0], i[1])
            cv2.circle(plot_img, center, 1, (255, 0 ,0), 3)
            radius = i[2]
            cv2.circle(plot_img, center, radius, (0, 255 ,0), 3)

        cv2.putText(plot_img, f"Frame {idx}", (25,25), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,255,0), 2)

        # for i in all_anchors[0]:
        #     center = (int(i[0]), int(i[1]))
        #     cv2.circle(plot_img, center, 1, (0, 0 ,255), 3)
        #     radius = i[2]
        #     cv2.circle(plot_img, center, int(radius), (0, 255 ,0), 3)
    idx += 1
    cv2.imshow('frame', plot_img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
means = np.mean(all_anchors, axis = 0)
for i in means:
    center = (int(i[0]), int(i[1]))
    cv2.circle(plot_img, center, 1, (0, 0 ,255), 3)
    radius = i[2]
    cv2.circle(plot_img, center, int(radius), (0, 0, 255), 3)
cv2.putText(plot_img, "Captured:", (25,25), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,255,0), 2)
cv2.imshow('frame', plot_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

plot_img = frame.copy()
means = np.mean(all_anchors, axis = 0)

In [ ]:
for i in means:
    center = (int(i[0]), int(i[1]))
    cv2.circle(plot_img, center, 1, (0, 0 ,255), 3)
    radius = i[2]
    cv2.circle(plot_img, center, int(radius), (0, 0, 255), 3)
cv2.putText(plot_img, "Captured:", (25,25), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,255,0), 2)
cv2.imshow('frame', plot_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [2]:
circles

NameError: name 'circles' is not defined

In [11]:
np.mean(all_anchors, axis = 0)

array([[ 273.86407767,   62.03883495,   33.26213592],
       [ 275.37864078, 1125.10679612,   33.2038835 ],
       [1336.25242718,   58.73786408,   33.5631068 ],
       [1341.26213592, 1123.06796117,   33.26213592]])

In [7]:
for i in all_anchors[0]:
    center = (i[0], i[1])
    print(center)

(257.56558026373386, 67.36486517637968)
(255.11006325483322, 1132.670260362327)
(1322.123750768602, 69.9921855777502)
(1321.7842183411121, 1135.4694909602404)


In [5]:
all_anchors

[array([[ 261.40832698,   71.01608941,   34.45982273],
        [ 255.04681563, 1131.99019809,   33.31539715],
        [1321.47369292,   69.81050823,   33.01046419],
        [1323.36348719, 1134.39922538,   33.47042323]])]

In [36]:
sorted_anchors = []
for frame in all_anchors:
    maxx = sorted(np.squeeze(frame), key=lambda x: x[0])

    left = maxx[:2]
    right = maxx[-2:]

    l_sorty = sorted(left, key=lambda x: x[1])
    r_sorty = sorted(right, key=lambda x: x[1])

    sorted_anchors.append(np.vstack((l_sorty, r_sorty)))

In [38]:
np.mean(sorted_anchors,axis = 0)

array([[ 320.31578947,   65.05263158,   33.31578947],
       [ 336.42105263, 1133.89473684,   33.05263158],
       [1386.73684211,   47.78947368,   33.94736842],
       [1407.47368421, 1114.63157895,   33.15789474]])

In [32]:
maxx = sorted(np.squeeze(all_anchors[0]), key=lambda x: x[0])

left = maxx[:2]
right = maxx[-2:]

l_sorty = sorted(left, key=lambda x: x[1])
r_sorty = sorted(right, key=lambda x: x[1])

np.vstack((l_sorty, r_sorty))

array([[ 320,   64,   30],
       [ 336, 1134,   34],
       [1388,   46,   32],
       [1408, 1116,   34]], dtype=uint16)

In [3]:
cv2.imwrite('test.png', frame)

True

In [24]:
img = frame.copy()
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,55,7)
_,img = cv2.threshold(img,127,255,cv2.THRESH_BINARY)
kernel = np.ones((2,2),np.uint8)
closing = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
#res = cv2.Canny(closing, 5, 70, 3)

blurred = cv2.blur(closing, (7, 7))

detected_circles = cv2.HoughCircles(image=blurred,
                                    method=cv2.HOUGH_GRADIENT,
                                    dp=1.2,
                                    minDist=10,
                                    param1=50,
                                    param2=50,
                                    minRadius=0,
                                    maxRadius=300
                                    )

detected_circles = np.uint16(np.around(detected_circles))
pt = detected_circles[0][0]
a, b, r = pt
plot_img = frame.copy()
cv2.circle(plot_img, (a, b), r, (0, 255, 0), 2)
cv2.circle(plot_img, (a, b), 1, (255, 0, 0), 3)


#blurred = cv2.GaussianBlur(res, (5, 5), 0)
#closing = cv2.morphologyEx(closing, cv2.MORPH_CLOSE, kernel)
#closing = cv2.morphologyEx(closing, cv2.MORPH_CLOSE, kernel)

# opening = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)

cv2.imshow('frame',plot_img)
cv2.waitKey(0)
cv2.destroyAllWindows()
#mop up the dirt
# cv2.dilate(g, g, None, 1)
# cv2.erode(g, g, None, 1)

In [22]:
img = frame.copy()
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

blur = cv2.GaussianBlur(img,(3,3),0)
ret, thresh = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

#ret,thresh = cv2.threshold(img,127,255,cv2.THRESH_BINARY)
kernel = np.ones((3,3),np.uint8)
#closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
#opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
dilation = cv2.dilate(thresh,kernel,iterations = 3)

blurred = cv2.blur(dilation, (7, 7))
detected_circles = cv2.HoughCircles(image=blurred,
                                    method=cv2.HOUGH_GRADIENT,
                                    dp=1.2,
                                    minDist=500,
                                    param1=100,
                                    param2=50,
                                    minRadius=50,
                                    maxRadius=300
                                    )
res = cv2.Canny(erosion, 5, 70, 3)
detected_circles = np.uint16(np.around(detected_circles))
pt = detected_circles[0][0]
a, b, r = pt
plot_img = frame.copy()
cv2.circle(plot_img, (a, b), r, (0, 255, 0), 2)
cv2.circle(plot_img, (a, b), 1, (255, 0, 0), 3)

cv2.imshow('frame',blurred)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [23]:
img = frame.copy()
#img_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)


img_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

mask1 = cv2.inRange(img_hsv, (0,50,20), (5,255,255))
mask2 = cv2.inRange(img_hsv, (175,50,20), (180,255,255))
mask = cv2.bitwise_or(mask1, mask2)
kernel = np.ones((3,3),np.uint8)
mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

# mask1 = cv2.inRange(img_hsv, (0,50,20), (5,255,255))
# mask2 = cv2.inRange(img_hsv, (175,50,20), (180,255,255))
# mask = cv2.bitwise_or(mask1, mask2)

# kernel = np.ones((3,3),np.uint8)
# erosion = cv2.erode(mask1,kernel,iterations = 1)
# kernel = np.ones((2,2),np.uint8)
# dilation = cv2.dilate(erosion,kernel,iterations = 4)

# kernel = np.ones((2,2),np.uint8)
# mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=2)

cv2.imshow('frame', mask)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Well plate

In [12]:
cap = cv2.VideoCapture(0)
cap = cv_core.set_res(cap, (1024, 768))

while True:
    ret, frame = cap.read()

    img = frame.copy()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    # blur = cv2.GaussianBlur(thresh, (5,5), 0)
    # kernel = np.ones((4,4), np.uint8)
    # erosion = cv2.erode(blur, kernel, iterations=1)

    # kernel = np.ones((3,3),np.uint8)
    # #closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    # #opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    # dilation = cv2.dilate(thresh,kernel,iterations = 3)

    # blurred = cv2.blur(dilation, (7, 7))
    detected_circles = cv2.HoughCircles(image=gray,
                                        method=cv2.HOUGH_GRADIENT,
                                        dp=1,
                                        minDist=30,
                                        param1=100,
                                        param2=40,
                                        minRadius=1,
                                        maxRadius=30
                                        )

    if detected_circles is not None and detected_circles.shape[1] == 96:
        break
cap.release()
plot_img = frame.copy()
circles = np.uint16(np.around(detected_circles))
for i in circles[0,:]:
    center = (i[0], i[1])

    cv2.circle(plot_img, center, 1, (255, 0 ,0), 3)
    radius = i[2]
    cv2.circle(plot_img, center, radius, (0, 255 ,0), 3)

cv2.imshow('frame', plot_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [11]:
centers = circles[0][:,:-1]

In [12]:

cv2.imshow('frame', plot_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [14]:
img = frame.copy()

img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

blur = cv2.GaussianBlur(img,(3,3),0)
ret, thresh = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
kernel = np.ones((4,4),np.uint8)
erosion = cv2.erode(thresh,kernel,iterations = 1)
res = cv2.Canny(erosion, 5, 70, 3)
#kernel = np.ones((2,2),np.uint8)
#opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)
closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)

blurred = cv2.blur(res, (5, 5))

# contours, hierarchy = cv2.findContours(
#             blurred, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

detected_circles = cv2.HoughCircles(image=blurred,
                                        method=cv2.HOUGH_GRADIENT,
                                        dp=1.2,
                                        minDist=25,
                                        param1=50,
                                        param2=50,
                                        minRadius=10,
                                        maxRadius=30
                                        )

plot_img = frame.copy()
#with_contours = cv2.drawContours(plot_img, contours, -1,(0,255, 0),2)
circles_img = np.uint16(np.around(detected_circles))
for i in circles_img[0,:]:
    cv2.circle(plot_img,(i[0],i[1]),i[2],(0,255,0),2)
    cv2.circle(plot_img,(i[0],i[1]),2,(0,0,255),3)

cv2.imshow('frame', erosion)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
from skimage.feature import peak_local_max
from skimage.segmentation import watershed
from scipy import ndimage

# Load in image, convert to gray scale, and Otsu's threshold
image = frame.copy()
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

# Remove small noise by filtering using contour area
cnts = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]

for c in cnts:
    if cv2.contourArea(c) < 1000:
        cv2.drawContours(thresh,[c], 0, (0,0,0), -1)

cv2.imshow('thresh', thresh)
# Compute Euclidean distance from every binary pixel
# to the nearest zero pixel then find peaks
distance_map = ndimage.distance_transform_edt(thresh)
local_max = peak_local_max(distance_map, indices=False, min_distance=20, labels=thresh)

# Perform connected component analysis then apply Watershed
markers = ndimage.label(local_max, structure=np.ones((3, 3)))[0]
labels = watershed(-distance_map, markers, mask=thresh)

# Iterate through unique labels
for label in np.unique(labels):
    if label == 0:
        continue

    # Create a mask
    mask = np.zeros(gray.shape, dtype="uint8")
    mask[labels == label] = 255

    # Find contours and determine contour area
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    c = max(cnts, key=cv2.contourArea)
    cv2.drawContours(image, [c], -1, (36,255,12), -1)

cv2.imshow('image', image)
cv2.waitKey()

/tmp/ipykernel_13336/2487679775.py:22: FutureWarning: indices argument is deprecated and will be removed in version 0.20. To avoid this warning, please do not use the indices argument. Please see peak_local_max documentation for more details.
  local_max = peak_local_max(distance_map, indices=False, min_distance=20, labels=thresh)


113

: 

In [5]:
frame = cv2.imread('camera_data/raw/image0.png')

In [27]:
kernel_ = np.array([[-1,-1,-1],[-1,9,-1],[-1,-1,-1]])
#im = cv2.filter2D(frame,-2,kernel_)
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
ret,thresh = cv2.threshold(gray,125,255,cv2.THRESH_BINARY_INV)
kernel = np.ones((3,3),np.uint8)
#closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)#
# lab = cv2.cvtColor(frame,cv2.COLOR_BGR2LAB)
# l_channel, a1, b1 = cv2.split(lab)

# clache = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
# cl = clache.apply(l_channel)
# limg = cv2.merge((cl,a1,b1))
# enhanced_img = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
result = np.hstack((thresh, opening))
cv2.namedWindow('frame', cv2.WINDOW_NORMAL)
cv2.imshow('frame', result)
cv2.waitKey(0)
cv2.destroyAllWindows()


QObject::moveToThread: Current thread (0x56537ecb3a40) is not the object's thread (0x56537ffd63d0).
Cannot move to target thread (0x56537ecb3a40)

QObject::moveToThread: Current thread (0x56537ecb3a40) is not the object's thread (0x56537ffd63d0).
Cannot move to target thread (0x56537ecb3a40)

QObject::moveToThread: Current thread (0x56537ecb3a40) is not the object's thread (0x56537ffd63d0).
Cannot move to target thread (0x56537ecb3a40)

QObject::moveToThread: Current thread (0x56537ecb3a40) is not the object's thread (0x56537ffd63d0).
Cannot move to target thread (0x56537ecb3a40)

QObject::moveToThread: Current thread (0x56537ecb3a40) is not the object's thread (0x56537ffd63d0).
Cannot move to target thread (0x56537ecb3a40)

QObject::moveToThread: Current thread (0x56537ecb3a40) is not the object's thread (0x56537ffd63d0).
Cannot move to target thread (0x56537ecb3a40)

QObject::moveToThread: Current thread (0x56537ecb3a40) is not the object's thread (0x56537ffd63d0).
Cannot move to tar

In [6]:
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

#adjusted = cv2.convertScaleAbs(gray, alpha=1.5)

# hsv2 = cv2.cvtColor(adjusted, cv2.COLOR_BGR2HSV)

# mask1 = cv2.inRange(hsv2, (50,60,70), (60,255,255))
# #mask2 = cv2.inRange(adjusted, (175,50,20), (180,255,255))
# mask = cv2.bitwise_or(mask1, mask2)
#blur = cv2.GaussianBlur(gray,(11,11),0)
ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

#ret,thresh = cv2.threshold(gray,125,255,cv2.THRESH_BINARY)
#kernel = np.ones((3,3),np.uint8)
#closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
#opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)#
#dilation = cv2.dilate(thresh,kernel,iterations = 3)

#blur2 = cv2.blur(dilation, (7, 7))
#blur3 = cv2.blur(closing, (7, 7))

cv2.namedWindow('frame', cv2.WINDOW_NORMAL)
cv2.namedWindow('frame1', cv2.WINDOW_NORMAL)
cv2.imshow('frame', thresh)
cv2.imshow('frame1', gray)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [30]:
cap = cv2.VideoCapture(0)
cap = cv_core.set_res(cap, (1024, 768))

while True:
    ret, frame = cap.read()
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray,(3,3),0)
    ret,thresh = cv2.threshold(blur,100,255,cv2.THRESH_BINARY)
    #ret, thresh = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    kernel = np.ones((3,3),np.uint8)
    #closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    #opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    dilation = cv2.dilate(thresh,kernel,iterations = 3)

    blur2 = cv2.blur(dilation, (7, 7))
    detected_circles = cv2.HoughCircles(image=blur2,
                                        method=cv2.HOUGH_GRADIENT,
                                        dp=1.2,
                                        minDist=500,
                                        param1=100,
                                        param2=50,
                                        minRadius=50,
                                        maxRadius=300
                                        )

    plot_img = frame.copy()
#with_contours = cv2.drawContours(plot_img, contours, -1,(0,255, 0),2)
    circles_img = np.uint16(np.around(detected_circles))
    for i in circles_img[0,:]:
        cv2.circle(plot_img,(i[0],i[1]),i[2],(0,255,0),2)
        cv2.circle(plot_img,(i[0],i[1]),2,(0,0,255),3)


    cv2.imshow('frame', plot_img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [7]:

def on_change(val): pass

def callback(): pass


cap = cv2.VideoCapture(0)
cap = cv_core.set_res(cap, camera_res['1200'])

cv2.namedWindow('frame',  cv2.WINDOW_NORMAL)
cv2.createTrackbar('slider', 'frame', 0, 255, on_change)
#cv2.createButton('Manual Lock', callback, None, cv2.QT_NEW_BUTTONBAR, 1)
cv2.resizeWindow('frame', 1348, 1011)

while True:
    ret, frame = cap.read()
    #val = cv2.getTrackbarPos('slider', 'frame')
    
    if ret:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        adjusted = cv2.convertScaleAbs(hsv, alpha=3)
        mask = cv2.inRange(adjusted, (0,100,255), (35,255,255))
        kernel = np.ones((5,5),np.uint8)
        opening = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
        dilation = cv2.dilate(opening, np.ones((3,3),np.uint8),iterations = 2)
        cv2.imshow('frame', dilation)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()